# COLLABORATIVE FILTERING USINGTHE NETFLIX DATA

# Problem 2: Analyzing the Netflix Data

Get the dataset from s3 bucket

In [1]:
import os
dbfs_dir = 's3://netflixdata-yr/'
movies_filename = dbfs_dir + 'movie_titles.txt'
testing_filename = dbfs_dir + 'TestingRatings.txt'
training_filename = dbfs_dir + 'TrainingRatings.txt'
# https://github.com/snehalnair/als-recommender-pyspark/blob/master/Recommendation_Engine_MovieLens.ipynb

Define the schemas

In [2]:
from pyspark.sql.types import *

movies_df_schema = StructType(
  [StructField('movie_Id', IntegerType()),
   StructField('year', IntegerType()),
   StructField('title', StringType())]
)

testing_df_schema = StructType(
  [StructField('movieId', IntegerType()),
   StructField('userId', IntegerType()),
   StructField('rating', DoubleType())]
)

training_df_schema = StructType(
  [StructField('movieId', IntegerType()),
   StructField('userId', IntegerType()),
   StructField('rating', DoubleType())]
)

Load and cache the data 

In [3]:
from pyspark.sql.functions import regexp_extract
from pyspark.sql.types import *

movies_titles_df = sqlContext.read.options(header=False, inferSchema=False).schema(movies_df_schema).csv("s3://netflixdata-yr/movie_titles.txt")
testing_ratings_df = sqlContext.read.options(header=False, inferSchema=False).schema(testing_df_schema).csv("s3://netflixdata-yr/TestingRatings.txt")
training_ratings_df = sqlContext.read.options(header=False, inferSchema=False).schema(testing_df_schema).csv("s3://netflixdata-yr/TrainingRatings.txt")

movies_titles_df.cache()
testing_ratings_df.cache()
training_ratings_df.cache()

DataFrame[movieId: int, userId: int, rating: double]

Print the 3 dataframes to view

In [5]:
print ('Movie Titles:')
movies_titles_df.show(3)
print ('Testing set:')
testing_ratings_df.show(3)
print ('Training set:')
training_ratings_df.show(3)

Movie Titles:


+--------+----+--------------------+
|movie_Id|year|               title|
+--------+----+--------------------+
|       1|2003|     Dinosaur Planet|
|       2|2004|Isle of Man TT 20...|
|       3|1997|           Character|
+--------+----+--------------------+
only showing top 3 rows

Testing set:


+-------+-------+------+
|movieId| userId|rating|
+-------+-------+------+
|      8| 573364|   1.0|
|      8|2149668|   3.0|
|      8|1089184|   3.0|
+-------+-------+------+
only showing top 3 rows

Training set:


[Stage 3:>                                                          (0 + 1) / 1]

+-------+-------+------+
|movieId| userId|rating|
+-------+-------+------+
|      8|1744889|   1.0|
|      8|1395430|   2.0|
|      8|1205593|   4.0|
+-------+-------+------+
only showing top 3 rows



Get the Shape of all the 3 DataFrames

In [6]:
print("Shape of movies set:",(movies_titles_df.count(), len(movies_titles_df.columns)))
print("Shape of testing set:",(testing_ratings_df.count(), len(testing_ratings_df.columns)))
print("Shape of training df:",(training_ratings_df.count(), len(training_ratings_df.columns)))

Shape of movies set: (17770, 3)
Shape of testing set: (100478, 3)


[Stage 10:=============================>                            (2 + 2) / 4]

Shape of training df: (3255352, 3)


 Merge testing and training dataframe with movie titles dataframe and drop the duplicate column movieID 

In [7]:
testing_df = movies_titles_df.join(testing_ratings_df, movies_titles_df["movie_Id"]==testing_ratings_df["movieId"])
testing_df = testing_df.drop("movieId")
print(testing_df.show(3))

+--------+----+--------------------+-------+------+
|movie_Id|year|               title| userId|rating|
+--------+----+--------------------+-------+------+
|       8|2004|What the #$*! Do ...| 573364|   1.0|
|       8|2004|What the #$*! Do ...|2149668|   3.0|
|       8|2004|What the #$*! Do ...|1089184|   3.0|
+--------+----+--------------------+-------+------+
only showing top 3 rows

None


In [8]:
training_df = movies_titles_df.join(training_ratings_df, movies_titles_df["movie_Id"]==training_ratings_df["movieId"])
training_df = training_df.drop("movieId")
print(training_df.show(3))

+--------+----+--------------------+-------+------+
|movie_Id|year|               title| userId|rating|
+--------+----+--------------------+-------+------+
|       8|2004|What the #$*! Do ...|1744889|   1.0|
|       8|2004|What the #$*! Do ...|1395430|   2.0|
|       8|2004|What the #$*! Do ...|1205593|   4.0|
+--------+----+--------------------+-------+------+
only showing top 3 rows

None


Get average ratings for all the movies

In [9]:
from pyspark.sql.functions import col, avg, when, count
avg_rating= training_df.groupby("title").avg("rating").alias("avg_rating")
avg_rating.show(3)

[Stage 18:==============>                                           (1 + 3) / 4]

+--------------------+------------------+
|               title|       avg(rating)|
+--------------------+------------------+
|       The Big Tease| 3.112189859762675|
|Police Academy 3:...|3.4135966578047854|
|David Blaine: Fea...|3.4537444933920707|
+--------------------+------------------+
only showing top 3 rows



21/12/14 16:58:32 WARN Utils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.debug.maxToStringFields' in SparkEnv.conf.


Creating temp view for analysis

In [10]:
from pyspark.sql import SQLContext
import pyspark
sqlContext = pyspark.SQLContext(sc)  
testing_df.createOrReplaceTempView('testing_df')
training_df.createOrReplaceTempView('training_df')

# Problem 2: 

How many distinct items(movie_id) & how many distinct users are there in the test set (TestingRatings.txt)?

In [11]:
from pyspark.sql.functions import countDistinct
items_test=testing_df.select(countDistinct("movie_Id"))
items_test.show()

users_test=testing_df.select(countDistinct("userId"))
users_test.show()

+------------------------+
|count(DISTINCT movie_Id)|
+------------------------+
|                    1701|
+------------------------+

+----------------------+
|count(DISTINCT userId)|
+----------------------+
|                 27555|
+----------------------+



In [12]:
items_test=training_df.select(countDistinct("movie_Id"))
items_test.show()

users_test=training_df.select(countDistinct("userId"))
users_test.show()

+------------------------+
|count(DISTINCT movie_Id)|
+------------------------+
|                    1821|
+------------------------+



[Stage 43:>                                                         (0 + 4) / 4]

+----------------------+
|count(DISTINCT userId)|
+----------------------+
|                 28978|
+----------------------+



Pick a user (user_Id=79 from the test set, extract the items this user has rated in the training set

In [13]:
user_data = (training_df[training_df.userId == '79'])
user_data.show(3)
print(user_data.groupBy("userId").agg({'title': 'count'}).show(3))
"""User 79 picked from testing set has rated 84 movies in the training set"""

+--------+----+--------------------+------+------+
|movie_Id|year|               title|userId|rating|
+--------+----+--------------------+------+------+
|     361|2004|The Phantom of th...|    79|   4.0|
|     636|1980|          Stir Crazy|    79|   3.0|
|     851|1990|Back to the Futur...|    79|   4.0|
+--------+----+--------------------+------+------+
only showing top 3 rows

+------+------------+
|userId|count(title)|
+------+------------+
|    79|          84|
+------+------------+

None


'User 79 picked from testing set has rated 84 movies in the training set'

Get the items user 79 has rated in the training set as a list

In [14]:
distinct_movies =user_data.select("title").rdd.flatMap(lambda x: x).collect()
print(distinct_movies)

['The Phantom of the Opera: Special Edition', 'Stir Crazy', 'Back to the Future Part III', 'Uptown Girls', "National Lampoon's Vacation", 'Hook', 'No Way Out', 'Belly of the Beast', 'The American President', 'Beverly Hills Cop', 'Undertow', 'MVP: Most Valuable Primate', 'When Harry Met Sally', 'Message in a Bottle', 'Napoleon Dynamite', 'Sister Act 2: Back in the Habit', 'Beaches', 'History of the World: Part 1', 'Bad Boys II', 'Confessions of a Teenage Drama Queen', 'The Italian Job', '3 Ninjas', 'Rain Man', 'The Cannonball Run', 'Taps', "Mask: Director's Cut", 'Sleeping With the Enemy', 'Guess Who', 'Star Trek IV: The Voyage Home', 'The Boy Who Could Fly', 'Some Kind of Wonderful', 'Catch That Kid', 'Pretty Woman', "Big Momma's House", 'Good Morning', 'Gremlins', "Ferris Bueller's Day Off", "Charlie's Angels: Full Throttle", 'Blade', 'The World Is Not Enough', 'Stepmom', 'As Good as It Gets', 'Soccer Dog', 'Splash: 20th Anniversary Edition', 'Ice Princess', "Can't Buy Me Love", 'The 

Compute how many other users in the training set have rated the same items(movie titles)

In [15]:
from pyspark.sql.functions import col, avg, when, count
import pyspark.sql.functions as F
from pyspark.sql.functions import countDistinct

user_training=training_df.filter(F.col("title").isin(distinct_movies))
print(user_training.show(3))

+--------+----+--------------------+-------+------+
|movie_Id|year|               title| userId|rating|
+--------+----+--------------------+-------+------+
|     361|2004|The Phantom of th...|2569320|   4.0|
|     361|2004|The Phantom of th...|1987434|   2.0|
|     361|2004|The Phantom of th...| 946314|   2.0|
+--------+----+--------------------+-------+------+
only showing top 3 rows

None


In [16]:
count_movie=user_training.groupBy("userId").agg(F.count('title').alias("cntMovie"))
count_movie.show(3)

[Stage 62:>                                                         (0 + 4) / 4]

+-------+--------+
| userId|cntMovie|
+-------+--------+
| 128389|      59|
|2496050|      19|
| 455334|      37|
+-------+--------+
only showing top 3 rows



In [17]:
from pyspark.sql.functions import explode, col, udf, mean as _mean

movie_count = count_movie.select(
    _mean(col('cntMovie')).alias('mean'),
    ).collect()

mean_movie = movie_count[0]['mean']
print("Estimated average overlap of items for users",mean_movie)

Estimated average overlap of items for users 35.1158465042446


Pick a item (movie_Id=8 from the test set, extract the users that has rated this item in the training set

In [18]:
item_data = training_df[training_df['movie_Id']=='8']
print(item_data.show(3))

print(item_data.groupBy("movie_Id").agg({'userId': 'count'}).show(3))
"""For the movie_Id 8 in testing set 2381 users has rated the same movie in the training set"""

+--------+----+--------------------+-------+------+
|movie_Id|year|               title| userId|rating|
+--------+----+--------------------+-------+------+
|       8|2004|What the #$*! Do ...|1744889|   1.0|
|       8|2004|What the #$*! Do ...|1395430|   2.0|
|       8|2004|What the #$*! Do ...|1205593|   4.0|
+--------+----+--------------------+-------+------+
only showing top 3 rows

None
+--------+-------------+
|movie_Id|count(userId)|
+--------+-------------+
|       8|         2831|
+--------+-------------+

None


'For the movie_Id 8 in testing set 2381 users has rated the same movie in the training set'

In [19]:
distinct_items =item_data.select('userId').rdd.flatMap(lambda x: x).collect()
print(distinct_items)

[1744889, 1395430, 1205593, 1488844, 1447354, 306466, 1331154, 1818178, 991725, 1987434, 1765381, 433803, 1148143, 1174811, 1684516, 754781, 567025, 1623132, 1567095, 1666394, 622194, 966375, 116582, 1646405, 174868, 327122, 135348, 944189, 1041552, 2459251, 2327803, 440723, 1826612, 1574266, 2105990, 1515871, 1339724, 2161899, 46036, 885929, 180200, 603277, 2551764, 1951144, 1977925, 2162424, 2637831, 1620735, 1863499, 2239227, 361407, 227469, 1854947, 2030564, 411537, 581766, 2004175, 452192, 2002276, 2459804, 2439795, 228190, 1265877, 61863, 2628763, 21983, 958382, 2570542, 248932, 2605003, 459486, 1117379, 1378970, 489962, 360830, 19106, 922922, 322178, 2059716, 419330, 1007809, 319119, 1333, 1113647, 2530840, 793754, 675390, 1097350, 2416619, 2125035, 2127805, 2577867, 2541025, 1478381, 2543231, 221091, 1874317, 212729, 880866, 324026, 1249686, 1684328, 2397231, 2268573, 2475820, 2286538, 69779, 513501, 1784146, 1037313, 854454, 1616406, 1888772, 1931966, 2622904, 1630930, 1645535

get the other items (movies) rated by the users who also rated movie_id: 8 (What the #$*! Do ...)

In [20]:
from pyspark.sql.functions import col, avg, when,count
import pyspark.sql.functions as F
from pyspark.sql.functions import countDistinct

item_training=training_df.filter(F.col("userId").isin(distinct_items))
print(item_training.show(3))

+--------+----+--------------------+-------+------+
|movie_Id|year|               title| userId|rating|
+--------+----+--------------------+-------+------+
|       8|2004|What the #$*! Do ...|1744889|   1.0|
|       8|2004|What the #$*! Do ...|1395430|   2.0|
|       8|2004|What the #$*! Do ...|1205593|   4.0|
+--------+----+--------------------+-------+------+
only showing top 3 rows

None


count the users who rates the movies that were rated by the users who also rated movie_id: 8 (What the #$*! Do ...)

In [21]:
count_item=user_training.groupBy("title").agg(F.count('userId').alias("cntUser"))
count_item.show(3)

+------------------+-------+
|             title|cntUser|
+------------------+-------+
|As Good as It Gets|  23184|
| View from the Top|   7965|
|       Bad Boys II|  15996|
+------------------+-------+
only showing top 3 rows



Get the average user count  for  movies that were rated by the users who also rated movie_id: 8 (What the #$*! Do ...)

In [22]:
from pyspark.sql.functions import explode, col, udf, mean as _mean

user_count = count_item.select(
    _mean(col('cntUser')).alias('mean'),
    ).collect()

mean_item = user_count[0]['mean']
print("Estimated average overlap of user for items",mean_item)

Estimated average overlap of user for items 12114.130952380952


# Calulating for Sparsity for Training & Testing datasets for Ratings

In [23]:
#TRAINING SET SPARSTY
# Get the Count the total number of ratings in the training & testing datasets
total_train_count = training_df.select("rating").count()
total_test_count=testing_df.select("rating").count()

# get the Count the number of distinct user and distinct movies in the training & testing datasets
num_users_train = training_df.select("userId").distinct().count()
num_movies_train = training_df.select("movie_Id").distinct().count()
num_users_test = testing_df.select("userId").distinct().count()
num_movies_test = testing_df.select("movie_Id").distinct().count()

# Calculate sparsity
sparsity_train = (1.0 - (total_train_count *1.0)/(num_users_train * num_movies_train))*100
sparsity_test = (1.0 - (total_test_count *1.0)/(num_users_test * num_movies_test))*100
print("The ratings training set is ", "%.2f" % sparsity_train + "% empty.")
print("The ratings testing set is ", "%.2f" % sparsity_test + "% empty.")

The ratings training set is  93.83% empty.
The ratings testing set is  99.79% empty.


DIMENSIONS OF THE DATASETS: The number of distinct movies in the testing dataset was 1701, whereas the number of distinct users were 27555 The number of distinct movies in the training dataset was 1821 & whereas the number of distinct users were 28978 As per the information in description of the files: The training set has 3.25 million ratings & the test set has 100,000.
Shape of movies set: (17770, 3)
Shape of testing set: (100478, 3)
Shape of training df: (3255352, 3)

SPARSITY TESTING: The sparse the movie-user matrix is calculated by the percentage of zero values in the data. Calculating the sparsity level of ratings shows 93.83% of ratings in training data and 99.97% in testing data are missing, which in turn, indicates that the vast majority of entries in both datasets are zero. This is beacause the vast majority of movies receive very few or even no ratings at all by users.

MODEL CHOICE: Considering both Efficiency and Prediction Quality:

Based on the calculation of estimated average overlap of items for users and estimated average overlap of users for items, the choice will be to implementation either a KNN based item based or user based collaborative filtering movie recommender.

PROBLEM IN CHOOSING KNN collaborative filtering movie recommender model: The problem in choosing the KNN collaborative filtering movie recommender model for analysis is that it will not be accurate as the distance between similar items or opposite items will be both very large due to the very big sparsity scores. Additionally, since almost 99.97% of user-movie interactions are not yet recorded in the training set there will not be sufficient information for the system to make reliable inferences for users or items, in turn, causing Cold Start problem in KNN collaborative filtering movie recommender system.

The 3 major issues are:

Popularity Bias, and 2. Cold-Sart, and 3. Scalability Issue:
Popularity Bias: The issue of Popularity Bias arises when only a small number ofitems receive an extremely high number of interactions and due to the presence of unpopular items or items that only receive a fraction of interactions when compared to the highly popular ones.

Cold Start: A collaborative algorithm cannot recommend the item if there are no interactions are available, and the quality of recommendations will be very poor in the case of availability of few interactions.

Scalability Issue: Scalability is also a big issue in KNN modeldue to its time complexity is O(nd + kn), where n is the cardinality of the training set and d the dimension of each sample. KNN takes more time in making inference than training, thus increasing the prediction latency.

Marix factorization solve these issues as Model learns to factorize rating matrix into user and movie representations, which allows model to predict better personalized movie ratings for users With matrix factorization, less-known movies can have rich latent representations as much as popular movies have, which improves recommender’s ability to recommend less-known movies

Alternating Least Square (ALS) is a matrix factorization algorithm that is implemented in Apache Spark ML and built for a large-scale collaborative filtering problems as it runs itself in a parallel fashion. ALS can solve scalability and sparseness of the Ratings data.